In [3]:
import pandas as pd
import cufflinks as cf
import plotly.graph_objects as go
from plotly.offline import iplot
from IPython.display import Image
import plotly.offline as pyo
cf.go_offline()

In [4]:
df_PGT = pd.read_excel('C:/Users/Administrador/Desktop/tabelas/STG_PGT.xlsx')

In [5]:
def del_duplicates (x):
    x = x.drop_duplicates()
    return x
df_PGT = del_duplicates (df_PGT)
df_PGT

,ID_STG_PGT,VLR_PGT_FAT,DAT_VCT,COD_MDL,QTD_CLI_CAD_POS,QTD_PGT,ID_FNT_ITT,DES_TIP_PSS,DAT_RSS_FNT_ITT,DAT_INC_DBO
0,451,515711,16082022,A04,22,22,67,F,2020-01-16,2020-01-17 17:50:42.746
1,452,60763,26102020,A99,2,2,67,F,2020-01-16,2020-01-17 17:50:42.746
2,453,8588195,24022019,G01,22,22,67,F,2020-01-16,2020-01-17 17:50:42.746
3,454,369084,24062018,B99,16,23,67,F,2020-01-16,2020-01-17 17:50:42.746
4,455,13054,20022021,B99,1,1,67,F,2020-01-16,2020-01-17 17:50:42.746
...,...,...,...,...,...,...,...,...,...,...
15441,15422,795535,23012023,A04,47,48,67,F,2020-01-16,2020-01-17 17:50:42.746
15442,15423,519,20012018,A99,1,1,67,F,2020-01-16,2020-01-17 17:50:42.746
15443,15424,140534,26062020,A01,3,3,67,F,2020-01-16,2020-01-17 17:50:42.746
15444,15425,141887,11112024,A01,7,9,67,F,2020-01-16,2020-01-17 17:50:42.746


In [6]:
#Contabiliza valores nulos 
nulo_id_PGT = df_PGT['ID_STG_PGT'].isnull().sum()
nulo_VLR_FAT = df_PGT['VLR_PGT_FAT'].isnull().sum()
nulo_DAT_VCT = df_PGT['DAT_VCT'].isnull().sum()
nulo_MOD = df_PGT['COD_MDL'].isnull().sum()
nulo_DES_PESS = df_PGT['DES_TIP_PSS'].isnull().sum()

#visualização dos dados nulos
nulo_total = pd.DataFrame([nulo_id_PGT, nulo_VLR_FAT, nulo_DAT_VCT, nulo_MOD, nulo_DES_PESS], 
                          index=['ID da fonte',
                                'Valor pago consolidado',
                                'Datas de vencimento',
                                'Código da modalidade',
                                'Tipo de cliente'],
                          columns=['Quantidade de dados nulos'])
nulo_total

,Quantidade de dados nulos
ID da fonte,0
Valor pago consolidado,0
Datas de vencimento,0
Código da modalidade,0
Tipo de cliente,0


In [7]:
# Dados Totais da Operações
val_total_VLR = df_PGT['VLR_PGT_FAT'].sum()
val_total_qtd_cli_cad_pos = df_PGT['QTD_CLI_CAD_POS'].sum()
val_total_qtd_PGT = df_PGT['QTD_PGT'].sum()


tot_OPR = pd.DataFrame([val_total_VLR,val_total_qtd_cli_cad_pos,val_total_qtd_PGT], 
                          index=['Valor Total pago consolidado',
                                'Quantidade de CPFs/CNPJs distintos',
                                'Quantidade de pagamento que foram disponibilizados'],
                          columns=['Valores Totais da Remessa'])
tot_OPR

,Valores Totais da Remessa
Valor Total pago consolidado,2393937320155
Quantidade de CPFs/CNPJs distintos,37795967
Quantidade de pagamento que foram disponibilizados,50805683


In [21]:
#Tabela: Modalidade do pagamento
categoria2 = df_PGT.groupby(by='COD_MDL').size()
print (categoria2)
grafico_categoria2 =categoria2.iplot(asFigure=True, kind="barh",title = "Código da modalidade",color = "orange")
grafico_categoria2.show()

COD_MDL
A01    3260
A04    3581
A05     503
A99    1126
B01      88
B03     637
B05    2460
B06     125
B07     209
B99    2386
C01      93
D01     523
F01       2
G01     453
dtype: int64


In [8]:
#Verifica o tipo do cliente (Física - F ou Jurídica - J) e contabiliza erro, se estiver fora da adequação

pes_fis_PGT = 0
pes_jur_PGT = 0
erro_pes_PGT = 0

for index, row in df_PGT.iterrows():
    if str(row['DES_TIP_PSS']).upper() == "F":
        pes_fis_PGT +=1
    elif str(row['DES_TIP_PSS']).upper() == "J":
        pes_jur_PGT +=1
    else:
        erro_pes_PGT +=1
        
        
tip_cli_PGT = pd.DataFrame([pes_fis_PGT,pes_jur_PGT,erro_pes_PGT], 
                          index=['Pessoa Física',
                                'Pessoa Jurídica',
                                'Erro'],
                          columns=['Quantidade'])
tip_cli_PGT

,Quantidade
Pessoa Física,15446
Pessoa Jurídica,0
Erro,0


In [23]:
#Visualização gráfica do Tipo de Cliente
labels_PGT = ['Pessoa Física','Pessoa Jurídica','Erro']
values_PGT = [pes_fis_PGT, pes_jur_PGT, erro_pes_PGT]

fig2 = go.Figure(data=[go.Pie(labels=labels_PGT, values=values_PGT,textinfo='label+percent',
                             insidetextorientation='radial'
                            )])
fig2.update_layout(title_text="Tipo de Cliente")

fig2.show()

In [9]:
#Completa com zeros à esquerda as datas de vencimento
df_PGT['DAT_VCT'] = df_PGT['DAT_VCT'].astype(str)
width = 8
df_PGT["DAT_VCT"]= df_PGT["DAT_VCT"].str.zfill(width) 

In [10]:
#Criação de uma coluna com somente o ano da data de vencimento
df_PGT['Ano'] = df_PGT['DAT_VCT'].apply(lambda x: str(x)[4:])

In [11]:
#Verifica se todas as datas possuem 8 dígitos
df_PGT['Qtd de dígitos'] = df_PGT['DAT_VCT'].apply(len) #Criação de uma coluna com a quantidade de dígitos
df_PGT['Qtd de dígitos'].value_counts()

8    15446
Name: Qtd de dígitos, dtype: int64

In [12]:
ano = df_PGT.groupby(by='Ano').size()
ano_dict  = ano.to_dict()
year = [i for i in ano_dict.keys()]
values = [j for j in ano_dict.values()]
data = ano_dict

In [13]:
fig2 = go.Figure(
   data=[
       go.Scatter(
           x=year,
           y=values,
           mode='lines+markers' 
       )
   ],
   layout=go.Layout(
       title='Datas de vencimento separadas por ano'
   )
)
pyo.plot(fig2)

fig2.show()

In [16]:
#Função para selecionar a modalidade de acordo com a data de vencimento
def mod_dat_vct (modalidade):
    return df_PGT[df_PGT['COD_MDL'] == modalidade]['Ano'].value_counts()

In [15]:
#Verificar a modalidade A01
mod_dat_vct ('A01')

2019    350
2020    289
2021    289
2022    285
2023    277
2024    275
2018    267
2025    256
2026    233
2017    197
2016    194
2027    179
2015    166
2028      3
Name: Ano, dtype: int64

In [17]:
#Verificar a modalidade B01
mod_dat_vct ('B01')

2019    82
2018     6
Name: Ano, dtype: int64